In [1]:
# Set up the environment
import os
import pandas as pd
from sqlalchemy import create_engine

In [2]:
# Code from Tier 1: LocalSQLConnection.py
import sqlite3
from sqlite3 import Error
 
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
 
    return conn
 
def select_all_tasks(conn):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    
#     query1 = """
#         SELECT *
#         FROM FACILITIES
#         """

#     query1 = """
#     SELECT f.name, 
#     SUM(CASE WHEN b.memid =0 THEN guestcost
#         ELSE membercost END) AS total_revenue
#     FROM Bookings AS b
#     LEFT JOIN Facilities AS f 
#     ON b.facid = f.facid
#     GROUP BY f.name
#     HAVING total_revenue <1000
#     ORDER BY total_revenue 
#     """

#     query1 = '''
#     SELECT 
#         m.surname||" "||m.firstname AS member,
#         r.surname||" "||r.firstname AS recommender
#     FROM Members as m
#     LEFT JOIN Members as r
#     ON r.memid = m.recommendedby
#     ORDER BY member;'''
        
#     query1 = '''
#     SELECT
#         f.name,
#         COUNT (b.memid) AS member_usage
#     FROM Bookings AS b
#     LEFT JOIN Facilities AS f
#     ON b.facid = f.facid
#     WHERE b.memid != 0
#     GROUP BY f.name
#     '''
    query1 = '''
    SELECT
        f.name,
        strftime('%m', b.starttime) AS month,
        COUNT (b.memid) AS member_usage
    FROM Bookings AS b
    LEFT JOIN Facilities AS f
    ON b.facid = f.facid
    WHERE b.memid != 0
    GROUP BY f.name, month
    '''
    cur.execute(query1)
 
    rows = cur.fetchall()
    headers = [i[0] for i in cur.description]
    print(headers)
    for row in rows:
        print(row)


def main():
    database = "sqlite_db_pythonsqlite.db"
 
    # create a database connection
    conn = create_connection(database)
    with conn: 
        print("2. Query all tasks")
        select_all_tasks(conn)
 
 
if __name__ == '__main__':
    main()

2.6.0
2. Query all tasks
['name', 'month', 'member_usage']
('Badminton Court', '07', 51)
('Badminton Court', '08', 132)
('Badminton Court', '09', 161)
('Massage Room 1', '07', 77)
('Massage Room 1', '08', 153)
('Massage Room 1', '09', 191)
('Massage Room 2', '07', 4)
('Massage Room 2', '08', 9)
('Massage Room 2', '09', 14)
('Pool Table', '07', 103)
('Pool Table', '08', 272)
('Pool Table', '09', 408)
('Snooker Table', '07', 68)
('Snooker Table', '08', 154)
('Snooker Table', '09', 199)
('Squash Court', '07', 23)
('Squash Court', '08', 85)
('Squash Court', '09', 87)
('Table Tennis', '07', 48)
('Table Tennis', '08', 143)
('Table Tennis', '09', 194)
('Tennis Court 1', '07', 65)
('Tennis Court 1', '08', 111)
('Tennis Court 1', '09', 132)
('Tennis Court 2', '07', 41)
('Tennis Court 2', '08', 109)
('Tennis Court 2', '09', 126)


In [3]:
# Query list of facilities with total revenue of less than 1000
# Output facility name and total revenue, sorted by revenue
queryRevenue = '''
SELECT f.name, 
    SUM(CASE WHEN b.memid =0 THEN guestcost
        ELSE membercost END) AS total_revenue
FROM Bookings AS b
LEFT JOIN Facilities AS f 
ON b.facid = f.facid
GROUP BY f.name
HAVING total_revenue <1000
ORDER BY total_revenue 
'''

In [4]:
# Create engine to CountryClub database
engine = create_engine('sqlite:///sqlite_db_pythonsqlite.db')
with engine.connect () as con:
    rs = con.execute(queryRevenue)
    revenueDF = pd.DataFrame(rs.fetchall())
    revenueDF.columns = rs.keys()

revenueDF.head()

,name,total_revenue
0,Table Tennis,90.0
1,Snooker Table,115.0
2,Pool Table,265.0
3,Badminton Court,604.5


In [5]:
# Produce a report of members and who recommended them in alphabetic surname,firstname order */
queryMemberRecommend = '''
    SELECT 
        m.surname||" "||m.firstname AS member,
        r.surname||" "||r.firstname AS recommender
    FROM Members as m
    LEFT JOIN Members as r
    ON r.memid = m.recommendedby
    ORDER BY member;'''

In [6]:
# Create engine to CountryClub database
engine = create_engine('sqlite:///sqlite_db_pythonsqlite.db')
with engine.connect () as con:
    rs = con.execute(queryMemberRecommend)
    recommendDF = pd.DataFrame(rs.fetchall())
    recommendDF.columns = rs.keys()

print(recommendDF)

                     member        recommender
0            Bader Florence    Stibbons Ponder
1                Baker Anne    Stibbons Ponder
2             Baker Timothy     Farrell Jemima
3                Boothe Tim         Rownam Tim
4            Butters Gerald       Smith Darren
5               Coplin Joan      Baker Timothy
6             Crumpet Erica        Smith Tracy
7                Dare Nancy    Joplette Janice
8             Farrell David               None
9            Farrell Jemima               None
10              GUEST GUEST               None
11          Genting Matthew     Butters Gerald
12                Hunt John  Purview Millicent
13              Jones David    Joplette Janice
14            Jones Douglas        Jones David
15          Joplette Janice       Smith Darren
16           Mackenzie Anna       Smith Darren
17             Owen Charles       Smith Darren
18             Pinker David     Farrell Jemima
19        Purview Millicent        Smith Tracy
20           

In [7]:
# Q12: Find the facilities with their usage by member, but not guests
queryFacilityMemberUse = '''
SELECT
    f.name,
    COUNT (b.memid) AS member_usage
FROM Bookings AS b
LEFT JOIN Facilities AS f
ON b.facid = f.facid
WHERE b.memid != 0
GROUP BY f.name
'''

In [8]:
# Create engine to CountryClub database
engine = create_engine('sqlite:///sqlite_db_pythonsqlite.db')
with engine.connect () as con:
    rs = con.execute(queryFacilityMemberUse)
    facilityDF = pd.DataFrame(rs.fetchall())
    facilityDF.columns = rs.keys()

print(facilityDF)

              name  member_usage
0  Badminton Court           344
1   Massage Room 1           421
2   Massage Room 2            27
3       Pool Table           783
4    Snooker Table           421
5     Squash Court           195
6     Table Tennis           385
7   Tennis Court 1           308
8   Tennis Court 2           276


In [9]:
# Q13: Find the facilities usage by month, but not guests
queryFacilityMonthUsage = '''
SELECT
    f.name,
    strftime('%m', b.starttime) AS month,
    COUNT (b.memid) AS member_usage
FROM Bookings AS b
LEFT JOIN Facilities AS f
ON b.facid = f.facid
WHERE b.memid != 0
GROUP BY f.name, month
'''

In [10]:
# Create engine to CountryClub database
engine = create_engine('sqlite:///sqlite_db_pythonsqlite.db')
with engine.connect () as con:
    rs = con.execute(queryFacilityMonthUsage)
    facilityMonthDF = pd.DataFrame(rs.fetchall())
    facilityMonthDF.columns = rs.keys()

print(facilityMonthDF)

               name month  member_usage
0   Badminton Court    07            51
1   Badminton Court    08           132
2   Badminton Court    09           161
3    Massage Room 1    07            77
4    Massage Room 1    08           153
5    Massage Room 1    09           191
6    Massage Room 2    07             4
7    Massage Room 2    08             9
8    Massage Room 2    09            14
9        Pool Table    07           103
10       Pool Table    08           272
11       Pool Table    09           408
12    Snooker Table    07            68
13    Snooker Table    08           154
14    Snooker Table    09           199
15     Squash Court    07            23
16     Squash Court    08            85
17     Squash Court    09            87
18     Table Tennis    07            48
19     Table Tennis    08           143
20     Table Tennis    09           194
21   Tennis Court 1    07            65
22   Tennis Court 1    08           111
23   Tennis Court 1    09           132
